In [1]:
# Import necessary libraries
from datasets import load_dataset, Audio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa
import torch
import jiwer

/Users/jylee/Desktop/supreme-parakeet/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ensure that the directory is formatted as follows for parsing purposes:
# cv-valid-train/
# ├── metadata.csv (renamed cv-valid-train.csv: column 'filename' changed to 'file_name')
# └── cv-valid-train/
#     ├── sample-000000.mp3
#     ├── sample-000001.mp3
#     ...
data_dir = '/Users/jylee/Downloads/common_voice/cv-valid-train' # Path to common_voice/cv-valid-train/ directory

In [3]:
# Load the dataset
dataset = load_dataset('audiofolder', data_dir=data_dir, split='train')

# Preprocessing

In [14]:
# Cast audio to 16kHz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [5]:
# View a sample
dataset[0]

{'audio': {'path': '/Users/jylee/Downloads/common_voice/cv-valid-train/cv-valid-train/sample-000000.mp3',
  'array': array([ 0.00000000e+00, -2.39675413e-13, -2.89490112e-14, ...,
          4.10622626e-04,  7.94679625e-04,  7.57522997e-04]),
  'sampling_rate': 48000},
 'text': 'learn to recognize omens and follow them the old king had said',
 'up_votes': 1,
 'down_votes': 0,
 'age': None,
 'gender': None,
 'accent': None,
 'duration': None}

Wav2Vec2 was trained on audio data only, so for consistency, other data columns are removed.

In [17]:
# Remove the 'up_votes', 'down_votes', 'age', 'gender', 'accent', and 'duration' columns
dataset = dataset.remove_columns(
    ['up_votes', 'down_votes', 'age', 'gender', 'accent', 'duration']
)

In [23]:
# Create train-validation split
split_dataset = dataset.train_test_split(test_size=0.3, seed=42)
split_dataset

In CTC, it is common to classify speech chunks into letters. Extract all letters from the data and build vocabulary.

In [25]:
# Function to extract characters
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = split_dataset['train'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True)

Map:   0%|          | 0/137043 [00:02<?, ? examples/s]


ArrowInvalid: Column 2 named vocab expected length 137043 but got length 1